In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import seaborn as sns
import math

In [2]:
df = pd.read_excel("./pre_reg_data.xlsx")
df_a = pd.read_excel('./data/hhi_a.xlsx')
df_b = pd.read_excel('./data/hhi_b.xlsx')

In [18]:
df

,code,name,time,PPE,K,L,E,sales,COGS,gross,...,ln_l,ln_material,ln_invest,ln_ppe,hhi_a,ln_tfp,tfp,tfp_2,fs_op,tfp_op
0,1101,台泥,2000/12,49256658.0,93096593.0,53191441.0,39905152.0,17965395.0,17072199.0,893196.0,...,7.325808,13.430265,15.626880,15.890390,0.011225,-1.655132,0.191067,0.036507,-1.636886,0.194585
1,1101,台泥,2001/12,49947168.0,98029907.0,53887357.0,44142550.0,20244735.0,19819285.0,425450.0,...,7.286192,13.451313,15.051837,14.694238,0.005488,-2.403544,0.090397,0.008172,-2.298020,0.100458
2,1101,台泥,2002/12,48110722.0,101027776.0,52981497.0,48046279.0,24595720.0,23605540.0,990180.0,...,7.263330,13.519280,14.742414,13.263540,0.015391,-1.562871,0.209534,0.043904,-1.431390,0.238977
3,1101,台泥,2003/12,46635752.0,94574148.0,45686416.0,48887732.0,24420321.0,22821323.0,1598998.0,...,7.221836,13.388557,12.102721,13.580631,0.025643,-1.028041,0.357707,0.127954,-0.892413,0.409666
4,1101,台泥,2004/12,44671493.0,94158098.0,42052635.0,52105463.0,24197991.0,21539174.0,2658817.0,...,7.058758,13.824409,13.253055,11.898691,0.036996,-0.434135,0.647825,0.419677,-0.353446,0.702264
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46314,L2301,光寶,2001/12,1721534.0,16688990.0,7270161.0,9418829.0,15334837.0,12519730.0,2815107.0,...,6.342122,12.569764,12.726748,12.656871,0.240268,0.890907,2.437339,5.940622,0.988873,2.688205
46315,R073,和茂,2000/12,78102.0,251964.0,97309.0,154655.0,63154.0,41030.0,22124.0,...,3.850147,9.688313,10.797390,11.076372,0.000007,-0.497530,0.608031,0.369702,-0.310760,0.732889
46316,R073,和茂,2001/12,81889.0,294608.0,173455.0,121153.0,110713.0,82606.0,28107.0,...,3.970292,10.009468,11.472103,10.084100,0.000024,-0.400978,0.669665,0.448451,-0.420688,0.656595
46317,R176,錦星,2000/12,79173.0,868979.0,682571.0,186408.0,1686890.0,1554271.0,132619.0,...,3.737670,NaN,NaN,6.952729,0.025913,0.704660,2.023158,4.093170,0.654088,1.923388


In [4]:
## compare different va measurement

fig = go.Figure()
fig.add_trace(go.Scatter(y = df.loc[(df['rank_a'] <= 20) & (df['rank_a'] != 0) & (df['industry'] == 2)]['va'], mode='lines', name=f'mearsure 1'))
fig.add_trace(go.Scatter(y = df.loc[(df['rank_a'] <= 20) & (df['rank_a'] != 0) & (df['industry'] == 2)]['va_3'], mode='lines', name=f'mearsure 2'))
fig.add_trace(go.Scatter(y = df.loc[(df['rank_a'] <= 20) & (df['rank_a'] != 0) & (df['industry'] == 2)]['va_1'], mode='lines', name=f'mearsure 3'))

fig.update_layout(
    title = 'Three Value Added Measurement', 
    yaxis_title = "Value Added",
)

fig.show()

In [4]:
## list out top 20 firms

for j in range(0, 6):
    industry = [2, 3, 4, 5, 6, 8][j]
    rank_yr = pd.DataFrame()

    for i in range(2000, 2024):
        temp = df.loc[(df['time'] == str(i)+'/12') & (df['rank_a'] <= 20) & (df['rank_a'] != 0) & (df['industry'] == industry)][['name', 'time', 'rank_a']]
        temp = temp.sort_values('rank_a').reset_index(drop=True)
        rank_yr[i] = temp['name']

    rank_yr.to_excel(f'top20_firm_data/top20_firms_industry{industry}.xlsx', index=False)

In [5]:
## list out labor share in each industry by rank

ls_df = pd.DataFrame()
ls_df['year'] = range(2002, 2024)

for j in range(0, 6):
    industry = [2, 3, 4, 5, 6, 8][j]
    
    for top in [5, 10, 20, 30, 'all']:
        ls_li = []

        for i in range(2002, 2024):
            if top != 'all':
                pay = df.loc[(df['time'] == str(i)+'/12') & (df['rank_a'] <= top) & (df['rank_a'] != 0) & (df['industry'] == industry)]['pay'].sum()
                va = df.loc[(df['time'] == str(i)+'/12') & (df['rank_a'] <= top) & (df['rank_a'] != 0) & (df['industry'] == industry)]['gross'].sum()
            else:
                pay = df.loc[(df['time'] == str(i)+'/12') & (df['industry'] == industry)]['pay'].sum()
                va = df.loc[(df['time'] == str(i)+'/12') & (df['industry'] == industry)]['gross'].sum()

            ls_li.append(pay/va)
        
        ls_df[f'industry{industry}_{top}'] = ls_li

ls_df

,year,industry2_5,industry2_10,industry2_20,industry2_30,industry2_all,industry3_5,industry3_10,industry3_20,industry3_30,...,industry6_5,industry6_10,industry6_20,industry6_30,industry6_all,industry8_5,industry8_10,industry8_20,industry8_30,industry8_all
0,2002,0.338017,0.320261,0.331629,0.325468,0.476304,0.265243,0.266489,0.273422,0.274944,...,0.573718,0.590099,0.631994,0.666135,0.671265,0.368507,0.490225,0.530058,0.529724,0.646455
1,2003,0.271543,0.256761,0.271877,0.286743,0.456250,0.279060,0.284759,0.298865,0.302074,...,0.669910,0.709357,0.716480,0.760970,0.764753,0.560434,0.695947,0.630486,0.600394,0.680823
2,2004,0.176545,0.201286,0.209024,0.235276,0.372619,0.310965,0.313620,0.322599,0.326920,...,0.494733,0.498671,0.528878,0.538331,0.557551,0.346570,0.414069,0.360365,0.454333,0.509737
3,2005,0.168665,0.196891,0.252944,0.259556,0.429015,0.305601,0.313097,0.319162,0.327188,...,0.608755,0.611044,0.643184,0.662762,0.665953,0.387827,0.440835,0.450117,0.425197,0.481331
4,2006,0.147305,0.185516,0.229457,0.235678,0.379350,0.320080,0.323192,0.340158,0.347611,...,0.947077,0.962146,0.968457,1.007657,1.039604,0.118358,0.246837,0.275834,0.341991,0.402167
5,2007,0.153732,0.171624,0.206276,0.213351,0.360905,0.318739,0.328812,0.327363,0.335004,...,0.884803,0.816483,0.813327,0.834684,1.423177,0.116245,0.138412,0.334819,0.320207,0.360316
6,2008,0.279950,0.281455,0.345561,0.347762,0.592381,0.308014,0.323246,0.329172,0.333581,...,1.748431,1.462860,1.443227,1.676935,5.334304,0.142268,0.228761,0.403427,0.392752,0.439609
7,2009,0.214653,0.249014,0.333146,0.330464,0.580340,0.317508,0.326625,0.339153,0.342939,...,1.170397,1.094764,1.078167,1.110566,2.416769,0.273152,0.260567,0.356815,0.361086,0.395127
8,2010,0.233948,0.255307,0.288347,0.320958,0.458776,0.336274,0.346520,0.347660,0.351148,...,0.410600,0.441031,0.460889,0.467673,0.473771,0.045846,0.244838,0.342877,0.327668,0.358297
9,2011,0.233191,0.264919,0.323352,0.338406,0.629812,0.344910,0.356049,0.358836,0.365127,...,0.893913,0.924164,0.942230,0.990242,1.417785,0.142730,0.156497,0.326430,0.344843,0.354897


In [6]:
## manufacturing labor share by rank

fig = go.Figure()

for i in [5, 10, 20, 30, 'all']:

    fig.add_trace(go.Scatter(x = ls_df['year'], y = ls_df[f'industry2_{i}'],
                    mode='lines',
                    name=f'industry2_{i}'))
    
fig.update_layout(
    title = 'Labor share in manufacturing 2002-2023', 
    xaxis_title = "Year",
    yaxis_title = "labor share",
    xaxis = dict(
        tickmode = 'linear',
        tick0 = 2002,
        dtick = 1
    ),

    yaxis = dict(
        tickmode = 'linear',
        tick0 = 0,
        dtick = 0.1
    )
)

fig.show()

In [7]:
## utilites and commumication labor share by rank

fig = go.Figure()

for i in [5, 10, 20, 30, 'all']:

    fig.add_trace(go.Scatter(x = ls_df['year'], y = ls_df[f'industry3_{i}'],
                    mode='lines',
                    name=f'industry3_{i}'))
    
fig.update_layout(
    title = 'Labor share in uti&commu 2002-2023', 
    xaxis_title = "Year",
    yaxis_title = "labor share",
    xaxis = dict(
        tickmode = 'linear',
        tick0 = 2002,
        dtick = 1
    ),

    yaxis = dict(
        tickmode = 'linear',
        tick0 = 0,
        dtick = 0.1
    )
)

fig.show()

In [8]:
## construction labor share by rank

fig = go.Figure()

for i in [5, 10, 20, 30, 'all']:

    fig.add_trace(go.Scatter(x = ls_df['year'], y = ls_df[f'industry4_{i}'],
                    mode='lines',
                    name=f'industry4_{i}'))
    
fig.update_layout(
    title = 'Labor share in construction 2002-2023', 
    xaxis_title = "Year",
    yaxis_title = "labor share",
    xaxis = dict(
        tickmode = 'linear',
        tick0 = 2002,
        dtick = 1
    ),

    yaxis = dict(
        tickmode = 'linear',
        tick0 = 0,
        dtick = 0.1
    )
)

fig.show()

In [9]:
## retail labor share by rank

fig = go.Figure()

for i in [5, 10, 20, 30, 'all']:

    fig.add_trace(go.Scatter(x = ls_df['year'], y = ls_df[f'industry5_{i}'],
                    mode='lines',
                    name=f'industry5_{i}'))
    
fig.update_layout(
    title = 'Labor share in retail 2002-2023', 
    xaxis_title = "Year",
    yaxis_title = "labor share",
    xaxis = dict(
        tickmode = 'linear',
        tick0 = 2002,
        dtick = 1
    ),

    yaxis = dict(
        tickmode = 'linear',
        tick0 = 0,
        dtick = 0.1
    )
)

fig.show()

In [10]:
## tranportation labor share by rank

fig = go.Figure()

for i in [5, 10, 20, 30, 'all']:

    fig.add_trace(go.Scatter(x = ls_df['year'], y = ls_df[f'industry6_{i}'],
                    mode='lines',
                    name=f'industry6_{i}'))
    
fig.update_layout(
    title = 'Labor share in trans 2002-2023', 
    xaxis_title = "Year",
    yaxis_title = "labor share",
    xaxis = dict(
        tickmode = 'linear',
        tick0 = 2002,
        dtick = 1
    ),

    yaxis = dict(
        tickmode = 'linear',
        tick0 = 0,
        dtick = 0.1
    )
)

fig.show()

In [11]:
## services labor share by rank

fig = go.Figure()

for i in [5, 10, 20, 30, 'all']:

    fig.add_trace(go.Scatter(x = ls_df['year'], y = ls_df[f'industry8_{i}'],
                    mode='lines',
                    name=f'industry8_{i}'))
    
fig.update_layout(
    title = 'Labor share in service 2002-2023', 
    xaxis_title = "Year",
    yaxis_title = "labor share",
    xaxis = dict(
        tickmode = 'linear',
        tick0 = 2002,
        dtick = 1
    ),

    yaxis = dict(
        tickmode = 'linear',
        tick0 = 0,
        dtick = 0.1
    )
)

fig.show()

In [68]:
## average value added of top 50 firms in manu 

temp = pd.DataFrame()
rank_li = []
avg_va_li = []

for i in range(101):
    rank_li.append(i)
    avg_va_li.append(df.loc[(df['industry'] == 2) & (df['rank_a'] == i)]['va'].mean())

temp['rank'] = rank_li
temp['avg_va'] = avg_va_li
temp = temp[temp['rank'] != 0]

fig = px.bar(temp[:50], x = "rank", y = "avg_va", title="Average Value Added in Manufacturing")
fig.show()

In [69]:
## average market share of top 50 firms in manu 

temp = pd.DataFrame()
rank_li = []
avg_va_li = []

for i in range(101):
    rank_li.append(i)
    avg_va_li.append(df.loc[(df['industry'] == 2) & (df['rank_a'] == i)]['cons_a'].mean())

temp['rank'] = rank_li
temp['avg_mk'] = avg_va_li
temp = temp[temp['rank'] != 0]

fig = px.bar(temp[:50], x = "rank", y = "avg_mk", title="Average Market Share in Manufacturing")
fig.show()